In [1]:
import pandas as pd
import numpy as np
import datetime
from functools import reduce
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

### Project Overview

1. The data is incremental with respect to the date. The dataset is for full calender year of 2016 and 2017. 
2. Since the predictions are to be provided for 2 quarters of 2018, the data will be split from Jan 2016 upto June 2017 
for training and July 2017 to Dec 2017 to create target variable. So those employees leaving the company from July 2017 
to Dec 2017 will be labeled as 1. The training data will be filtered for all employees except those who have already 
churned before July 2017 because those such employees will neither be labelled as 1 nor 0.
3. The developed model will be used to predict Jan 2018 to June 2018 attrition, there will be a stride of 6 months 
in the training data. Thus the training data for prediction will be from July 2016 to Dec 2017. 

In [132]:
def data_preprocess(df, start_date, cut_off_date, last_date):
    '''
    inputs:
    dataset to be processed
    start_date: the start date of the 'LastWorkingDate' 
    cut_off_date: the end date of the training dataset 
    last_date: end date of validation/test dataset
    
    output:
    data: a merge of several dataframes engineered from raw dataset
    1. df_demograph: dataframe of unique values for demographic data
    2. df_salary_change: dataframe engineered from 'Salary'to derive the 'increment' column
    3. df_promotion: dataframe engineered from 'Joining Designation' and 'Designation' to derive 'Promotion'
    4. df_total: dataframe engineered to derive the total business-value of the employee
    5. df_average: dataframe engineered to derive the total business-value of the employee
    6. df_working_days:
    7. df_reporting:                                                                    
    8. df_target:                                                                   
                                                                        
    
    
    Promotion forms a significant part in establishing job performance for
    the employer and job satisfaction for the employee. Also if any increase in the designation was calculated based
    on joining designation and the designation.
    4. df_total: A part of Feature Engineering. It is the total business value generated by the hardworking employee
    throughout his tenure or the cutoff date
    5. df_average:  A part of Feature Engineering. average_business_value, average_salary and average_quarterly_rating 
    generated by the employee throughout his tenure or the cutoff date
    6. df_reporting: A part of Feature Engineering. Total reporting count of each employee
    7. df_working_days: A part of Feature Engineering. Total number of employment days for each employee. If the employee
    has last working day, then the days were counted from joining date to last working day else joining date to cutoff date
    
    Target varibale is created using cutoff date, to prepare the training data, cutoff date will be 1 July 2017 that is
    all employees who had attrition from 1 July 2017 to 31 Dec 2017 will be labeled as 1. Those who have not yet given
    their resignation till 1 July 2017 will be tagged as 0. 
       
    
    '''
    # Filtering data suitable for creating training data
    df = df[df['MMM-YY']>start_date]
    df = df[df['MMM-YY']<cut_off_date]    
#     df = df[(df['Dateofjoining']<cut_off_date)]  
    
    # Demographic dataset creation for each employee
    df_demograph = df[['Emp_ID', 
                   'Age', 
                   "Gender", 
                   "City", 
                   "Education_Level"]].groupby('Emp_ID').max().reset_index()
    
    # Feature Engineer - min_salary, max_salary and increment
    df_salary_min = df.groupby('Emp_ID').min()['Salary'].reset_index()
    df_salary_min.columns =['Emp_ID', 
                            'Salary_min']
    df_salary_max = df.groupby('Emp_ID').max()['Salary'].reset_index()
    df_salary_max.columns =['Emp_ID', 
                            'Salary_max']    
    df_salary_change = pd.merge(df_salary_min, 
                                df_salary_max, 
                                how = 'inner', 
                                on = 'Emp_ID')
    df_salary_change['Increment'] = ((df_salary_change['Salary_max'] - df_salary_change['Salary_min'])/df_salary_change['Salary_min'] *100).astype(int)
    
    # Feature Engineering - Promotion if there is any increase in the designation. The more the number, higher the promotion
    df_promotion = df
    df_promotion["promotion"] = np.where(df_promotion['Joining Designation']==df_promotion['Designation'], 0, 
                                     df_promotion['Designation'] - df_promotion['Joining Designation'])
    df_promotion = df_promotion[['Emp_ID',
                                 'Joining Designation', 
                                 'Designation', 
                                 'promotion']].groupby("Emp_ID").max().reset_index()
    
    # Feature Engineering - total business value generated by the employee throughout his tenure or the cutoff date
    df_total = df.groupby('Emp_ID').sum()
    df_total = df_total["Total Business Value"].reset_index()
    df_total = df_total.set_axis(['Emp_ID', 
                                  'total_business_value'], 
                                 axis=1, 
                                 inplace=False)
    
    # Feature Engineering - average business value, average salary and average quarterly rating generated by the 
    #employee throughout his tenure or the cutoff date
    
    df_average = df.groupby('Emp_ID').mean()
    df_average = df_average[['Salary', 
                             'Total Business Value', 
                             'Quarterly Rating']].reset_index()
    df_average = df_average.set_axis(['Emp_ID', 
                                      'avg_salary', 
                                      'avg_business_value', 
                                      'avg_quartely_rating'], 
                                     axis=1, 
                                     inplace=False)
    
    # Feature Engineering - total reporting count by each employee
    df_reporting = df.groupby(['Emp_ID']).count()[['MMM-YY']].reset_index()
    df_reporting.columns = ["Emp_ID", 
                            "total_reportings"]
    
    # Feature Engineering - total working days by each employee
    df_working_days = df.groupby(['Emp_ID']).max().reset_index()
    df_working_days['number_employment_days'] = np.where(df_working_days['LastWorkingDate']<cut_off_date, 
                                                          df_working_days['LastWorkingDate'] - df_working_days["Dateofjoining"],
                                                          cut_off_date - df_working_days["Dateofjoining"])
    df_working_days = df_working_days[['Emp_ID',  
                                       "number_employment_days"]]

    data = reduce(lambda x,y: pd.merge(x,y, on='Emp_ID', how='inner'), [df_demograph, 
                                                                        df_average, 
                                                                        df_working_days, 
                                                                        df_promotion, 
                                                                        df_salary_change, 
                                                                        df_total, 
                                                                        df_reporting])    
    data["business_value_index"] = round(data["total_business_value"]/data["number_employment_days"].dt.days.astype('int16'),2)
    return data

### Input data
The dataframe is loaded as well as three dates are to be supplied

1. start_date: This date will provide the starting date for the data to be preprocessed. 
    In the given project, the training data is for 1.5 years and target of 0.5 years. 
    The data will roll every 0.5 years for the new predictions. So the start date can be mentioned accordingly
2. cut_off_date: This date will provide the training data cutoff date, beyond which data will not be considered
3. last_date: This date provides the overall last date of the data

##### Exploratory analysis of demographic data on the basis of full data available. Thus the cutoff date considered here is 1 July 2017 and start_date is 31 Dec 2015. Thus attrition of all employees (irrespective of their lat working day) are considered

In [ ]:
# df = pd.read_csv('D:/History/AV/attrition_data_hackathon/data/train_MpHjUjU.csv')
# df[['MMM-YY','Dateofjoining', 'LastWorkingDate']] = df[['MMM-YY',
#                                                         'Dateofjoining', 
#                                                         'LastWorkingDate']].apply(pd.to_datetime, 
#                                                                                   format='%Y-%m-%d')
# start_date = np.datetime64(datetime.datetime(2015,12,31))
# cut_off_date = np.datetime64(datetime.datetime(2018,1,1))

In [ ]:
# data = data_preprocess(df, start_date, cut_off_date)

In [ ]:
# # sns.scatterplot(data['Age'], data['target'])
# plt.figure(figsize=(8,5))
# sns.catplot(x='target',y='Age',data=data)

In [ ]:
# pd.crosstab(data['target'],data['Gender'])

In [ ]:
# sns.heatmap(pd.crosstab(data['target'],data['Gender']))

In [ ]:
# pd.crosstab(data['target'],data['Education_Level'])

In [ ]:
# sns.heatmap(pd.crosstab(data['target'],data['Education_Level']))

In [ ]:
# (pd.crosstab(data['target'],data['City'])).transpose()

In [ ]:
# sns.heatmap(pd.crosstab(data['target'],data['City']))

### Observation
1. Age: Almost equal distribution between churn and non-churners
2. Gender: Both females and males have higher attrition rate
3. Education level: All levels has higher attrition rate
4. City: there are few cities C10, C12, C16, C22, C24, C26, C27, C29, C3, C5 who has almost equal distribution of churners vs non-churners

##### Conclusion: Thus cities needs to be one-hot coded which could have impact on attrition rate. Rest other demographic data may not be useful in predicting attrition rate

### Exploratory analysis of other features. 

The target variable will be created based on cutoff date 1 July 2017 and start_date is 31 Dec 2015. Thus attrition of employees post 1 July will be labelled as 1, while others were tagged as 0. The data will be filtered using cutoff date and hence employees whos attrition is less than 1 July will not be considered and those employees whos joining date is beyong 1 July will not be considered in the training data.

In [133]:
df = pd.read_csv('D:/History/AV/attrition_data_hackathon/data/train_MpHjUjU.csv')
df[['MMM-YY','Dateofjoining', 'LastWorkingDate']] = df[['MMM-YY',
                                                        'Dateofjoining', 
                                                        'LastWorkingDate']].apply(pd.to_datetime, 
                                                                                  format='%Y-%m-%d')
start_date = np.datetime64(datetime.datetime(2016,1,1))
cut_off_date = np.datetime64(datetime.datetime(2017,1,1))
last_date = np.datetime64(datetime.datetime(2017,7,1))

In [134]:
# Filtering data suitable for creating training data
df_target = df 
df_target = df[df['MMM-YY']>cut_off_date]# & (df['LastWorkingDate']<last_date)]
df_target = df[df['MMM-YY']<last_date]
df_target = df[df['Dateofjoining']<cut_off_date]
df_target.drop(df_target[df_target['LastWorkingDate']<cut_off_date].index, inplace=True)
df_target['target'] = np.where((df_target['LastWorkingDate']>cut_off_date) & (df_target['LastWorkingDate']<last_date), 1, 0)

df_target = df_target[['Emp_ID',
                        'Dateofjoining',
                        'LastWorkingDate',
                        'target']]

df_target = df_target.groupby('Emp_ID').max()
df_target['target'].value_counts()

C:\Users\Spice\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-134-802caf0bce7b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['target'] = np.where((df_target['LastWorkingDate']>cut_off_date) & (df_target['LastWorkingDate']<last_date), 1, 0)


0    1183
1     285
Name: target, dtype: int64

In [135]:
data = data_preprocess(df, start_date, cut_off_date, last_date)

In [136]:
data = pd.merge(data,df_target, on="Emp_ID")

In [144]:
data.columns

Index(['Emp_ID', 'Age', 'Gender', 'City', 'Education_Level', 'avg_salary',
       'avg_business_value', 'avg_quartely_rating', 'number_employment_days',
       'Joining Designation', 'Designation', 'promotion', 'Salary_min',
       'Salary_max', 'Increment', 'total_business_value', 'total_reportings',
       'business_value_index', 'Dateofjoining', 'LastWorkingDate', 'target'],
      dtype='object')

In [145]:
data.drop(['Age', 'Gender', 'Education_Level', 'Dateofjoining', 'LastWorkingDate', 'City'], axis = 1, inplace = True)

In [150]:
data = data[['Emp_ID', 'number_employment_days', 'Designation', 'promotion', 'Increment', 'business_value_index', 'target']]

In [ ]:
# profile_report = ProfileReport(data, title="Pandas Profiling Report")
# profile_report.to_widgets()

#### plots vs churn
<br> 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level'
<br> 'avg_business_value', 'avg_quartely_rating', 'number_employment_days'
<br> 'promotion'
<br> 'Increment'
<br> 'total_reportings'
<br> 'business_value_index'

In [ ]:
data.info()

### columns to be selected
avg_business_value, 
avg_quartely_rating,
Joining Designation,
Designation,
promotion,
Increment,
total_reportings,
business_value_index

In [ ]:
# multivariate analysis by scatterplot of avg_business_value, avg_salary and target showed that less the business_value, more is the churn
sns.scatterplot(data['avg_business_value'], data['avg_quartely_rating'], hue = data['target'])

In [ ]:
# low business_value_index is clearly associated with churners. That means people bringing low business value per working day are likely to be churned
sns.barplot(data['target'], data['business_value_index'])

In [ ]:
# low avg_salary can lead to churners

sns.barplot(data['churn'], data['avg_salary'])


In [ ]:
#total low reportings are associated with churners
sns.barplot(data['churn'], data['total_reportings'])

In [ ]:
#total low reportings are associated with churners
sns.barplot(data['churn'], data['avg_quartely_rating'])

In [ ]:
# low avg_salary can lead to churners

sns.heatmap(pd.crosstab(data['target'],data['promotion']))

In [ ]:
pd.crosstab(data['target'],data['promotion'])

In [ ]:
pd.crosstab(data['target'],data['Designation'])

In [ ]:
pd.crosstab(data['target'],data['Joining Designation'])

In [ ]:
pd.crosstab(data['target'],data['total_reportings'])

In [ ]:
pd.crosstab(data['target'],data['Increment'])

In [ ]:
pd.crosstab(data['target'],data['avg_quartely_rating'])

In [ ]:
sns.catplot(x="churn", y="avg_quartely_rating", data=data) #jitter=False,

In [ ]:
sns.catplot(x="churn", y="avg_quartely_rating", kind="swarm", data=data) #jitter=False,

In [138]:
data['number_employment_days'] = data['number_employment_days'].dt.days

In [7]:
data.drop(['Age', 'Gender', 'Education_Level', 'Dateofjoining', 'LastWorkingDate', 'City'], axis = 1, inplace = True)

In [139]:
data['business_value_index'] = data['business_value_index'].fillna(0)

In [ ]:
data = data[['Emp_ID', 'number_employment_days', 'Designation', 'promotion', 'Increment', 'business_value_index', 'target']]

In [140]:
data['target'].value_counts()

0    1179
1     282
Name: target, dtype: int64

In [152]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

# calculate_vif_(x_train, 5)

In [148]:
calculate_vif_(X,5)

C:\Users\Spice\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\Spice\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1687: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


dropping 'avg_salary' at index: 0
dropping 'Joining Designation' at index: 3
dropping 'Salary_min' at index: 5
dropping 'avg_business_value' at index: 0
dropping 'Salary_max' at index: 4
dropping 'avg_quartely_rating' at index: 0
dropping 'total_reportings' at index: 5
dropping 'total_business_value' at index: 4
Remaining variables:
Index(['number_employment_days', 'Designation', 'promotion', 'Increment',
       'business_value_index'],
      dtype='object')


,number_employment_days,Designation,promotion,Increment,business_value_index
0,78,1,0,0,-8531.79
1,25,2,0,0,0.00
2,58,1,0,0,2075.17
3,175,1,0,0,14898.17
4,1679,4,3,0,3668.69
...,...,...,...,...,...
1456,91,2,0,0,0.00
1457,92,1,0,0,0.00
1458,1539,3,1,0,4837.80
1459,419,2,0,0,6190.95


In [ ]:
# Train-test split

In [151]:
target='target'
primary_key = "Emp_ID"

X = data.drop([target, primary_key],1)
y = data[target]
# Divide the training data given to train and validation data frames
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [153]:
x_train.to_csv('D:/History/AV/attrition_data_hackathon/data/x_train.csv',index=False)
x_val.to_csv('D:/History/AV/attrition_data_hackathon/data/x_val.csv',index=False)
y_train.to_csv('D:/History/AV/attrition_data_hackathon/data/y_train.csv',index=False)
y_val.to_csv('D:/History/AV/attrition_data_hackathon/data/y_val.csv',index=False)
# x_test.to_csv('D:/History/AV/attrition_data_hackathon/data/processed_data/x_train.csv/x_test.csv',index=False)